In [1]:
# !pip install psycopg2

## Library Used

In [2]:
import pandas as pd
from sqlalchemy import create_engine, inspect
import psycopg2

## Load Data

### Load data from database

In [3]:
db_host='ep-red-bar-a1rghkov.ap-southeast-1.aws.neon.tech'
db_user='siswa_bfp'
db_pass='bfp_aksel_keren'
db_name='ecommerce_db'

url = f'postgresql://{db_user}:{db_pass}@{db_host}/{db_name}?sslmode=require'

engine = create_engine(url)
conn = engine.connect()

inspector = inspect(engine)
tables = inspector.get_table_names()

print(tables)

['customers', 'carts', 'cartitems', 'products']


In [4]:
engine.dispose()

In [5]:
query_customer = '''
SELECT * FROM customers
'''

query_carts = '''
SELECT * FROM carts
'''

query_cartitems = '''
SELECT * FROM cartitems
'''

query_products = ''' 
SELECT * FROM products
'''

df_customer = pd.read_sql(query_customer, conn)
df_carts = pd.read_sql(query_carts, conn)
df_cartitems = pd.read_sql(query_cartitems, conn)
df_products = pd.read_sql(query_products, conn)

In [6]:
df_customer.head(5)

,cust_id,name,phone,address,city,province
0,1,PT Nuraini Tbk,(070) 602-8828,Gang Tebet Barat Dalam No. 5,Bekasi,Jawa Barat
1,2,PT Hariyah,(0130) 156 5916,Gg. Ciwastra No. 3,Bekasi,Jawa Barat
2,3,CV Farida Budiman Tbk,081 016 2783,Gg. Kiaracondong No. 217,Jakarta Timur,Jakarta
3,4,Perum Firmansyah (Persero) Tbk,+62-034-551-1097,Jalan Antapani Lama No. 256,Jakarta Pusat,Jakarta
4,5,PD Saefullah Lestari Tbk,(0208) 127 0881,Gang Rajawali Barat No. 152,Jakarta Barat,Jakarta


In [7]:
df_carts.head(5)

,cart_id,cust_id,trx_date
0,1,3,2023-01-01 00:00:00
1,2,618,2023-01-01 00:00:00
2,3,942,2023-01-01 00:00:00
3,4,386,2023-01-01 00:00:00
4,5,812,2023-01-01 00:00:00


In [8]:
df_cartitems.head(5)

,cart_item_id,cart_id,product_id,order_amt
0,1,1,66,3
1,2,1,97,3
2,3,1,133,3
3,4,2,60,1
4,5,2,79,2


In [9]:
df_products.head(5)

,product_id,name,price
0,1,Japchae,72500
1,2,Frozen Tater Tots,35000
2,3,Frozen Hot Pockets,32000
3,4,Duck Foie Gras,112500
4,5,Mee Goreng,66000


### Load data from xlsx

In [10]:
url = 'https://raw.githubusercontent.com/Kurikulum-Sekolah-Pacmann/wrangling-bfp-aksel/refs/heads/main/ecommerce_data/Campaigns.xlsx'
df_campaigns = pd.read_excel(url, engine= 'openpyxl')
df_campaigns.head(5)

,campaign_id,name,start_date,campaign_type,campaign_objects
0,1,Festival Makanan Sehat,2023-01-01,Category,2;3;6
1,2,Bulan Cita Rasa Nusantara,2023-02-01,Product,8;45;64;113;114;120
2,3,Mei Festival Makanan Lokal,2023-05-01,Category,1;3;4
3,4,Festival Pedas Juli,2023-07-01,Category,3;4;5
4,5,Bulan Makanan Manis,2023-09-01,Category,1;2;3;5


### Load data from json

In [11]:
url = 'https://raw.githubusercontent.com/Kurikulum-Sekolah-Pacmann/wrangling-bfp-aksel/refs/heads/main/ecommerce_data/Products.json'
df_products_json = pd.read_json(url)
df_products_json.head(5)

,name,description,price,product_id,image_url,tags,category
0,Japchae,A Korean dish made with stir-fried sweet potat...,72500,1,https://fake.pacmann.io/img/1.jpg,"[potato, seasonings, stir-fried, korean, beef,...","{'name': 'noodles', 'category_id': 1}"
1,Frozen Tater Tots,"Small, grated potatoes formed into rounds and ...",35000,2,https://fake.pacmann.io/img/2.jpg,"[rounds, bites, grated, small, golden, into]","{'name': 'frozen', 'category_id': 2}"
2,Frozen Hot Pockets,Microwaveable pastries filled with a variety o...,32000,3,https://fake.pacmann.io/img/3.jpg,"[pepperoni, pastries, fillings]","{'name': 'frozen', 'category_id': 2}"
3,Duck Foie Gras,A luxurious French delicacy made from the live...,112500,4,https://fake.pacmann.io/img/4.jpg,"[luxurious, liver, terrine, delicacy, pâté]","{'name': 'chicken_duck', 'category_id': 3}"
4,Mee Goreng,A flavorful and spicy stir-fried noodle dish f...,66000,5,https://fake.pacmann.io/img/5.jpg,"[indonesia, yellow, stir-fried, sauces, noodle...","{'name': 'noodles', 'category_id': 1}"


## Transforming Data

In [12]:
products_dataframe = pd.DataFrame()
customers_dataframe = pd.DataFrame()
campaigns_dataframe = pd.DataFrame()
categories_dataframe = pd.DataFrame()
tags_dataframe = pd.DataFrame()
cart_items_dataframe = pd.DataFrame()
carts_dataframe = pd.DataFrame()
campaign_items_dataframe = pd.DataFrame()

### Product dataframe transform

#### check nan and duplicated val in df_products and df_products_json

In [13]:
print("Check products DF name :\n")
print(f"Product nan val : \n{df_products.isna().sum()}")
print("------------------------------------------")
print(f"Product duplicate id : {df_products.duplicated(subset= ['product_id', 'name']).any()}")
print("------------------------------------------")

print("Check products_json DF name :\n")
print(f"Product nan val : \n{df_products_json.isna().sum()}")
print("------------------------------------------")
print(f"Product duplicate id : {df_products_json.duplicated(subset= ['product_id', 'name']).any()}")
print("------------------------------------------")


Check products DF name :

Product nan val : 
product_id    0
name          0
price         0
dtype: int64
------------------------------------------
Product duplicate id : False
------------------------------------------
Check products_json DF name :

Product nan val : 
name           0
description    0
price          0
product_id     0
image_url      0
tags           0
category       0
dtype: int64
------------------------------------------
Product duplicate id : False
------------------------------------------


#### Products DF

In [14]:
products_dataframe = df_products.join(df_products_json, how= 'inner', rsuffix='x_')
products_dataframe.drop(columns=['namex_', 'pricex_', 'product_idx_', 'category', 'tags'], inplace= True)
products_dataframe.head(5)

,product_id,name,price,description,image_url
0,1,Japchae,72500,A Korean dish made with stir-fried sweet potat...,https://fake.pacmann.io/img/1.jpg
1,2,Frozen Tater Tots,35000,"Small, grated potatoes formed into rounds and ...",https://fake.pacmann.io/img/2.jpg
2,3,Frozen Hot Pockets,32000,Microwaveable pastries filled with a variety o...,https://fake.pacmann.io/img/3.jpg
3,4,Duck Foie Gras,112500,A luxurious French delicacy made from the live...,https://fake.pacmann.io/img/4.jpg
4,5,Mee Goreng,66000,A flavorful and spicy stir-fried noodle dish f...,https://fake.pacmann.io/img/5.jpg


#### Categories DF

In [15]:
prod_category_name = []
prod_category_id = []


for data in df_products_json['category']:
    prod_category_name.append(data['name'])
    prod_category_id.append(data['category_id'])
    
categories_dataframe['name'] = prod_category_name
categories_dataframe['category_id'] = prod_category_id
categories_dataframe

,name,category_id
0,noodles,1
1,frozen,2
2,frozen,2
3,chicken_duck,3
4,noodles,1
...,...,...
147,chicken_duck,3
148,noodles,1
149,frozen,2
150,beverages,5


#### Tags DF

In [16]:
tags_dataframe['product_id'] = df_products_json['product_id']
tags_dataframe['tag'] = df_products_json['tags']
tags_dataframe['tag_id'] = range(1, len(tags_dataframe) + 1)
tags_dataframe.set_index('tag_id', inplace= True)
tags_dataframe.reset_index(inplace= True)

tags_dataframe.head(5)

,tag_id,product_id,tag
0,1,1,"[potato, seasonings, stir-fried, korean, beef,..."
1,2,2,"[rounds, bites, grated, small, golden, into]"
2,3,3,"[pepperoni, pastries, fillings]"
3,4,4,"[luxurious, liver, terrine, delicacy, pâté]"
4,5,5,"[indonesia, yellow, stir-fried, sauces, noodle..."


#### Campaign DF

In [17]:
campaigns_dataframe = df_campaigns[['campaign_id', 'name', 'campaign_type', 'start_date']]
campaigns_dataframe.head(5)

,campaign_id,name,campaign_type,start_date
0,1,Festival Makanan Sehat,Category,2023-01-01
1,2,Bulan Cita Rasa Nusantara,Product,2023-02-01
2,3,Mei Festival Makanan Lokal,Category,2023-05-01
3,4,Festival Pedas Juli,Category,2023-07-01
4,5,Bulan Makanan Manis,Category,2023-09-01


#### Campaign Items DF

In [18]:
item = []

for index in df_campaigns['campaign_objects']:
    item.append(index.split(";"))

# print(item)
campaign_items_dataframe = df_campaigns.copy()
campaign_items_dataframe['campaign_objects'] = item
campaign_items_dataframe = campaign_items_dataframe.explode('campaign_objects').reset_index(drop= True)
campaign_items_dataframe.rename(columns={'campaign_objects' : 'campaign_item'}, inplace= True)
campaign_items_dataframe.drop(columns={'name', 'start_date'}, inplace= True)
campaign_items_dataframe['campaign_item_id'] = range(1, len(campaign_items_dataframe) + 1)
campaign_items_dataframe.set_index('campaign_item_id', inplace= True)
campaign_items_dataframe.reset_index(inplace= True)

campaign_items_dataframe.head(10)

,campaign_item_id,campaign_id,campaign_type,campaign_item
0,1,1,Category,2
1,2,1,Category,3
2,3,1,Category,6
3,4,2,Product,8
4,5,2,Product,45
5,6,2,Product,64
6,7,2,Product,113
7,8,2,Product,114
8,9,2,Product,120
9,10,3,Category,1


## Publish Data

In [33]:
# Convert data 
df_cartitems.to_csv('data_csv/cart_items.csv', index= False)
df_carts.to_csv('data_csv/carts.csv', index= False)
df_customer.to_csv('data_csv/customers.csv', index= False)

tags_dataframe.to_csv('data_csv/tags.csv', index= False)
products_dataframe.to_csv('data_csv/products.csv', index= False)
categories_dataframe.to_csv('data_csv/categories.csv',index= False)

campaigns_dataframe.to_csv('data_csv/campaigns.csv', index= False)
campaign_items_dataframe.to_csv('data_csv/campaign_items.csv', index= False)